In [0]:
import tensorflow as tf


In [0]:
#Reset Default graph - Needed only for Jupyter notebook
tf.reset_default_graph()

**Colect Data**

In [0]:
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [4]:
import io
from google.colab import drive
#drive.mount()
#df = pd.read_csv(io.StringIO(uploaded['prices.csv'].decode('utf-8')))
data = pd.read_csv('./prices.csv')
data.head(5)

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [5]:
data.dtypes

date       object
symbol     object
open      float64
close     float64
low       float64
high      float64
volume    float64
dtype: object

### Consider only first 1000 rows in the dataset for building feature set and target set


In [0]:
data_df = data.head(1000)

In [0]:
data_df = data_df.drop(['date', 'symbol'], axis = 1)

In [0]:
y = data_df['close']

In [0]:
data_df = data_df.drop(['close'], axis=1)

In [0]:
X = data_df

In [11]:
# Split  training and testing from data
X_train, X_test, y_train, y_test = train_test_split(data_df, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(800, 4) (800,)
(200, 4) (200,)


## Building the graph in tensorflow

In [0]:
# Define Input place holder
#Input features
x = tf.placeholder(shape=[None,4],dtype=tf.float32, name='x-input')

x_n = tf.layers.batch_normalization(x, training=True)

#Actual Prices
y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')

In [0]:
# Define Weights and Bias

W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]), name="Bias")

In [0]:
# Prediction 
y = tf.add(tf.matmul(x_n,W),b,name='output')

In [0]:
# Loss (Cost) function [Mean square error]

loss = tf.reduce_mean(tf.square(y-y_),name='Loss')

In [0]:
# GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]
train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

## Execute the Graph for 100 epochs and observe the loss


In [0]:
#Lets start graph Execution
sess = tf.Session()

# variables need to be initialized before we can use them
sess.run(tf.global_variables_initializer())

#how many times data need to be shown to model
training_epochs = 100

In [18]:
for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, train_loss = sess.run([train_op, loss],feed_dict={x:X_train, 
                                                        y_:y_train})
    
    if epoch % 10 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss)

Training loss at step:  0  is  8094.499
Training loss at step:  10  is  6535.2544
Training loss at step:  20  is  5068.888
Training loss at step:  30  is  3403.7832
Training loss at step:  40  is  3403.49
Training loss at step:  50  is  3403.4448
Training loss at step:  60  is  3403.4392
Training loss at step:  70  is  3403.441
Training loss at step:  80  is  3403.4417
Training loss at step:  90  is  3403.443


## **Get the shapes and values of W and b**
Hint: Use sess.run(W) to get W.

In [19]:
sess.run(y, feed_dict={x:X_train[0:1]})

array([[68.49016]], dtype=float32)

In [20]:
sess.run(W)

array([[-0.5865339 ],
       [-6.50038   ],
       [ 1.0161465 ],
       [-0.01796909]], dtype=float32)

In [21]:
sess.run(b)

array([24.177698], dtype=float32)

In [22]:
print (W.shape, b.shape)

(4, 1) (1,)


In [0]:
#sess.close()

In [0]:
_, test_loss = sess.run([train_op, loss],feed_dict={x:X_test, 
                                                        y_:y_test})
    

In [25]:
Abs_MS_Diff_loss = train_loss - test_loss
Abs_MS_Diff_loss

-9.675781

In [26]:
X.shape

(1000, 4)

### Linear Classification using Keras
Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
Use Mean square error as loss function and sgd as optimizer

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Reshape data from 2D to 1D -> 28x28 to 784
#model.add(tf.keras.layers.Reshape(4000,),input_shape=(1000,4))

#Normalize the data
model.add(tf.keras.layers.BatchNormalization(input_shape =(4,)))

#Add Dense Layer which provides 10 Outputs after applying softmax
model.add(tf.keras.layers.Dense(1))

#Comile the model
model.compile(optimizer='sgd', loss='mse')

In [28]:
model.fit(X_train, y_train, 
          validation_data=(X_test, y_test), 
          epochs=30,
          batch_size=X_train.shape[0])

Train on 800 samples, validate on 200 samples
Epoch 1/30
800/800 [==============================] - 0s 207us/step - loss: 8081.0552 - val_loss: 40.3213
Epoch 2/30
800/800 [==============================] - 0s 5us/step - loss: 7011.5225 - val_loss: 49722.1016
Epoch 3/30
800/800 [==============================] - 0s 5us/step - loss: 2388.5234 - val_loss: 686961.7500
Epoch 4/30
800/800 [==============================] - 0s 5us/step - loss: 4198.8613 - val_loss: 419.9042
Epoch 5/30
800/800 [==============================] - 0s 5us/step - loss: 5646.8408 - val_loss: 86709.3984
Epoch 6/30
800/800 [==============================] - 0s 5us/step - loss: 42.3445 - val_loss: 136202.9062
Epoch 7/30
800/800 [==============================] - 0s 5us/step - loss: 221.4760 - val_loss: 14874.3428
Epoch 8/30
800/800 [==============================] - 0s 6us/step - loss: 1355.9359 - val_loss: 251619.9219
Epoch 9/30
800/800 [==============================] - 0s 7us/step - loss: 4278.7930 - val_loss: 953.2

## Logistic Classification using Keras

In [45]:
# Load the Iris dataset
iris_data = pd.read_csv('./Iris.csv')
iris_data.head(5)
iris_data = pd.get_dummies(iris_data,prefix=['Species'])
iris_data.head(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0


In [46]:
iris_data.columns

Index(['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm',
       'Species_Iris-setosa', 'Species_Iris-versicolor',
       'Species_Iris-virginica'],
      dtype='object')

In [0]:
y = iris_data[['Species_Iris-setosa','Species_Iris-versicolor', 'Species_Iris-virginica']]
X = iris_data.drop(['Species_Iris-setosa','Species_Iris-versicolor', 'Species_Iris-virginica','Id'],axis =1)

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3)
print("\nX_train:\n")
print(X_train.head())
print(X_train.shape)

print("\nX_test:\n")
print(X_test.head())
print(X_test.shape)


X_train:

     SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm
72             6.3           2.5            4.9           1.5
64             5.6           2.9            3.6           1.3
133            6.3           2.8            5.1           1.5
43             5.0           3.5            1.6           0.6
74             6.4           2.9            4.3           1.3
(105, 4)

X_test:

     SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm
124            6.7           3.3            5.7           2.1
9              4.9           3.1            1.5           0.1
20             5.4           3.4            1.7           0.2
108            6.7           2.5            5.8           1.8
123            6.3           2.7            4.9           1.8
(45, 4)


## **Model**
# Build the model with following layers: **bold text**

First dense layer with 10 neurons with input shape 4 (according to the feature set) 

Second Dense layer with 8 neurons 

Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) 


Use SGD and categorical_crossentropy loss

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Reshape data from 2D to 1D -> 28x28 to 784
#model.add(tf.keras.layers.Reshape(4000,),input_shape=(1000,4))

#Normalize the data
model.add(tf.keras.layers.BatchNormalization(input_shape = (4,)))

#Add Dense Layer which provides 10 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3))

#Comile the model
model.compile(optimizer='sgd', loss='mse')

In [0]:
#Add 1st hidden layer
model.add(tf.keras.layers.Dense(10, activation='relu'))

In [0]:
#Add 2nd hidden layer
model.add(tf.keras.layers.Dense(8, activation='relu'))

In [0]:
#Add OUTPUT layer
model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [0]:
#Create optimizer with non-default learning rate
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)

#Compile the model
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [64]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_3 (Batch (None, 4)                 16        
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 15        
_________________________________________________________________
dense_6 (Dense)              (None, 10)                40        
_________________________________________________________________
dense_7 (Dense)              (None, 8)                 88        
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 27        
Total params: 186
Trainable params: 178
Non-trainable params: 8
_________________________________________________________________


In [68]:
model.fit(X_train,y_train,          
          validation_data=(X_test,y_test),
          epochs=30,
          batch_size=32)

Train on 105 samples, validate on 45 samples
Epoch 1/30
105/105 [==============================] - 0s 123us/step - loss: 0.4180 - acc: 0.8762 - val_loss: 0.5888 - val_acc: 0.6222
Epoch 2/30
105/105 [==============================] - 0s 170us/step - loss: 0.5318 - acc: 0.7810 - val_loss: 0.5420 - val_acc: 0.6444
Epoch 3/30
105/105 [==============================] - 0s 89us/step - loss: 0.3937 - acc: 0.8857 - val_loss: 0.5388 - val_acc: 0.6444
Epoch 4/30
105/105 [==============================] - 0s 105us/step - loss: 0.4043 - acc: 0.8857 - val_loss: 0.6099 - val_acc: 0.6222
Epoch 5/30
105/105 [==============================] - 0s 88us/step - loss: 0.3422 - acc: 0.8952 - val_loss: 0.5900 - val_acc: 0.6222
Epoch 6/30
105/105 [==============================] - 0s 130us/step - loss: 0.3544 - acc: 0.9048 - val_loss: 0.5383 - val_acc: 0.6444
Epoch 7/30
105/105 [==============================] - 0s 114us/step - loss: 0.3250 - acc: 0.9524 - val_loss: 0.5857 - val_acc: 0.6444
Epoch 8/30
105/105 

In [70]:
scores = model.evaluate(X_test,y_test)
scores

45/45 [==============================] - 0s 151us/step


[0.37115086449517143, 0.7777777830759685]